In [6]:
!git clone https://github.com/infamoussoap/ConvexHull.git

Cloning into 'ConvexHull'...
remote: Enumerating objects: 643, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 643 (delta 115), reused 172 (delta 72), pack-reused 428
Receiving objects: 100% (643/643), 1.06 MiB | 3.53 MiB/s, done.
Resolving deltas: 100% (338/338), done.


In [1]:
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.insert(0, 'ConvexHull')

In [3]:
from Optimizers.ConvexHull import CauchySimplex
from Optimizers.ConvexHull import EGD, PairwiseFrankWolfe
from Optimizers.ConvexHull import validate_stopping_conditions

In [4]:
def sample_unit_hypercube(N, dimension):
    """ N is the number of samples on each surface of the hypercube 
        
        https://stats.stackexchange.com/questions/504487/distributions-on-the-surface-of-a-hypercube
    """
    index = np.arange(dimension)
    X = np.zeros((N * dimension * 2, dimension))
    
    count = 0
    for k in range(dimension):  # The dimension of the surface to sample on
        working_indices = index[index != k]
        for j in range(2):  # The top or bottom face of the surface
            for i in range(N):
                X[count, working_indices] = np.random.rand(dimension - 1)
                X[count, k] = j
                
                count += 1
    return X

def generate_point_on_surface(d, top_surface, X):
    """ Generates a point sampled on the convex hull of X
    
        Parameters
        ----------
        d : int
            The dimension of the surface to sample from 
        top_surface : int
            0 for the bottom surface and 1 for the top surface
        X : np.ndarray
            The set of points
    """
    surface_points = X[X[:, d] == top_surface]
    n, _ = surface_points.shape
    
    w = np.random.rand(n)
    w = w / np.sum(w)
    
    return w @ surface_points

In [5]:
def test_optimizer(optimizer, w, y_true, tol=1e-5, max_iter=10_000, search_args=None):
    if search_args is None:
        search_args = {}
    
    start = time.time()
    for i in range(max_iter):
        w = optimizer.search(w, **search_args)
        
        if np.linalg.norm(w @ optimizer.X - y_true) < tol:
            break
    end = time.time()

    return i + 1, end - start, np.linalg.norm(w @ optimizer.X - y_true)

In [6]:
dimensions = [10, 15, 20, 25, 30, 35, 40, 45, 50]
num_trials = 50

samples_per_surface = 100

tol = 1e-5
max_iter = 10_000

In [7]:
for seed, dim in enumerate(dimensions):
    np.random.seed(seed)
    
    print(f"Starting {dim}")
    X = sample_unit_hypercube(samples_per_surface, dim)

    cs_results = pd.DataFrame(0.0, index=range(num_trials), columns=['Iterations', 'Time (sec)', 'Distance'])
    egd_results = pd.DataFrame(0.0, index=range(num_trials), columns=['Iterations', 'Time (sec)', 'Distance'])
    pfw_results = pd.DataFrame(0.0, index=range(num_trials), columns=['Iterations', 'Time (sec)', 'Distance'])

    for i in range(num_trials):
        # Generate Datapoint
        d, top_surface = np.random.randint(dim), np.random.randint(2)
        y_true = generate_point_on_surface(d, top_surface, X)

        y = y_true.copy()
        y[d] += (1 if top_surface else -1)

        # Test Cauchy Simplex
        optimizer = CauchySimplex(X, y)
        w = np.ones(len(X)) / len(X)
        cs_results.iloc[i, :] = test_optimizer(optimizer, w, y_true, tol=1e-5, max_iter=10_000, search_args={})

        # Test EGD
        optimizer = EGD(X, y)
        w = np.ones(len(X)) / len(X)
        egd_results.iloc[i, :] = test_optimizer(optimizer, w, y_true, tol=1e-5, max_iter=10_000, 
                                                search_args={'step_size': 10})

        # Test PFW
        optimizer = PairwiseFrankWolfe(X, y)
        w = np.zeros(len(X))
        w[0] = 1
        pfw_results.iloc[i, :] = test_optimizer(optimizer, w, y_true, tol=1e-5, max_iter=10_000, search_args={})
    
        # Save after every loop
        cs_results.to_csv(f"cs_{dim}.csv")
        egd_results.to_csv(f"egd_{dim}.csv")
        pfw_results.to_csv(f"pfw_{dim}.csv")

    print(f"Finished {dim}")

Starting 10
Finished 10
Starting 15
Finished 15
Starting 20
Finished 20
Starting 25
Finished 25
Starting 30
Finished 30
Starting 35
Finished 35
Starting 40
Finished 40
Starting 45
Finished 45
Starting 50
Finished 50
